In [1]:
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

import torch

sys.path.append("../RD-systems-and-test-benches/utils")
import model_fsolve as modf
import model_torch as modt

import scipy.optimize as sco
import scipy.interpolate as sci

import hx_hydraulic as hxhy
import from_excel as fe
import data_plots as dplt

import fluids as fds
from CoolProp.CoolProp import PropsSI

import openpyxl as opxl

In [2]:
path = r"G:\Drive partagés\BU04-Innovation\PVT-PL-model\Inputs"
file_name = "\V4.5_MPEx4.xlsx"
hx, par, cond = fe.initialize(path, file_name)

In [3]:
X0sol = np.array([1.58630099e+03, 1.58628546e+03, 1.58627887e+03, 1.58629695e+03,
       1.58635472e+03, 1.58646660e+03, 1.58664636e+03, 1.58690725e+03,
       1.58726198e+03, 1.58772280e+03, 1.58830148e+03, 1.58900942e+03,
       1.58985761e+03, 1.59085671e+03, 1.59201707e+03, 1.59334876e+03,
       1.59488532e+03, 1.59658887e+03, 1.59849252e+03, 1.60060548e+03,
       1.60293684e+03, 1.60549554e+03, 1.60829044e+03, 1.61133034e+03,
       1.61462398e+03, 1.61818008e+03, 1.62200736e+03, 1.62611457e+03,
       1.63051047e+03, 1.63520582e+03, 1.64020782e+03, 1.64552553e+03,
       1.65124081e+03, 1.65721767e+03, 1.66353827e+03, 1.67021232e+03,
       1.67724975e+03, 1.68466072e+03, 1.69245566e+03, 1.70064526e+03,
       1.70924055e+03, 1.71825285e+03, 1.72769387e+03, 1.73757566e+03,
       1.74791069e+03, 1.75871185e+03, 1.76999249e+03, 1.78176640e+03,
       1.79418446e+03, 1.80698841e+03, 1.82033026e+03, 1.83422598e+03,
       1.84869220e+03, 1.86374616e+03, 1.87940580e+03, 1.89568978e+03,
       1.91261748e+03, 1.93020910e+03, 1.94848561e+03, 1.96746890e+03,
       1.98836954e+03, 2.01121117e+03, 2.03839468e+03, 2.07469809e+03,
       1.31141532e-22, 2.46575697e+01, 4.84148072e+01, 7.13017751e+01,
       9.33473081e+01, 1.14579067e+02, 1.35023589e+02, 1.54706339e+02,
       1.73651756e+02, 1.91883299e+02, 2.09423488e+02, 2.26293952e+02,
       2.42515464e+02, 2.58107981e+02, 2.73090685e+02, 2.87482013e+02,
       3.01430899e+02, 3.14691955e+02, 3.27412891e+02, 3.39609511e+02,
       3.51297044e+02, 3.62490166e+02, 3.73203032e+02, 3.83449306e+02,
       3.93242180e+02, 4.02594411e+02, 4.11518336e+02, 4.20025903e+02,
       4.28128696e+02, 4.35837952e+02, 4.43164594e+02, 4.50119246e+02,
       4.56780850e+02, 4.63022323e+02, 4.68922125e+02, 4.74488238e+02,
       4.79731972e+02, 4.84662635e+02, 4.89289394e+02, 4.93621294e+02,
       4.97667281e+02, 5.01436224e+02, 5.04936936e+02, 5.08178197e+02,
       5.11168776e+02, 5.13917451e+02, 5.16433036e+02, 5.18724399e+02,
       5.20822742e+02, 5.22692639e+02, 5.24365489e+02, 5.25850635e+02,
       5.27157601e+02, 5.28296116e+02, 5.29276148e+02, 5.30107925e+02,
       5.30801971e+02, 5.31369131e+02, 5.31820604e+02, 5.32167976e+02,
       5.32423254e+02, 5.32598938e+02, 5.32707930e+02, 5.32763651e+02,
       1.02728407e-06, 1.01197506e-06, 9.97234534e-07, 9.83054489e-07,
       9.69427215e-07, 9.56345290e-07, 9.43801580e-07, 9.31789231e-07,
       9.20301669e-07, 9.09332599e-07, 8.98875997e-07, 8.88926111e-07,
       8.79477459e-07, 8.70524826e-07, 8.62063258e-07, 8.54088065e-07,
       8.46523075e-07, 8.39507570e-07, 8.32965879e-07, 8.26894341e-07,
       8.21289542e-07, 8.16148324e-07, 8.11467775e-07, 8.07245233e-07,
       8.03478280e-07, 8.00164745e-07, 7.97302703e-07, 7.94890469e-07,
       7.92926604e-07, 7.91411157e-07, 7.90342030e-07, 7.89718511e-07,
       7.89542849e-07, 7.89809368e-07, 7.90520800e-07, 7.91678513e-07,
       7.93281772e-07, 7.95331308e-07, 7.97828094e-07, 8.00773339e-07,
       8.04168497e-07, 8.08015261e-07, 8.12315566e-07, 8.17071593e-07,
       8.22285762e-07, 8.27960740e-07, 8.34099440e-07, 8.40705021e-07,
       8.47857204e-07, 8.55407027e-07, 8.63434745e-07, 8.71944517e-07,
       8.80940759e-07, 8.90428139e-07, 9.00411587e-07, 9.10896290e-07,
       9.21887698e-07, 9.33391524e-07, 9.45413749e-07, 9.57960621e-07,
       9.71831656e-07, 9.87033639e-07, 1.00516001e-06, 1.02939078e-06])

In [4]:
X0sol_tensor = torch.tensor(X0sol, requires_grad=True)

X0 = modf.initialize([],par,cond)
X0_tensor = torch.tensor(X0, requires_grad=True)
q_vect = X0[par['N']*2:par['N']*3]
q_tensor = torch.tensor(q_vect, requires_grad=True)

In [5]:
f = lambda X : modt.fun_wo_square(X,par,cond)

In [6]:
X0sol_tensor.is_leaf

True

In [7]:
variations = []
variations.append(X0sol_tensor)
percentage = 0.05  # 1% variation

for _ in range(10):
    variation = X0sol_tensor.clone()  # Clone the original tensor

    # Create an empty tensor for the updated values
    updated_variation = torch.empty_like(variation)

    for i in range(len(variation)):
        # Generate a random number 0 or 1
        rand_int = torch.randint(0, 2, (1,))

        # Map 0 to -1 and 1 to 1
        sign = -1 if rand_int.item() == 0 else 1

        # Perform the operation and assign to the new tensor
        updated_variation[i] = (1 + sign * percentage) * variation[i]

    # Replace the variation tensor with the updated tensor
    variation = updated_variation.clone().detach().requires_grad_(True)
    variations.append(variation)

In [8]:
fig = go.Figure()

loss_table = []

for i,tens in enumerate(variations):

    torch.autograd.grad(f(tens),tens)

    niters = 10

    losses = [f(tens).item()]

    optim = torch.optim.Adam([tens], lr=0.000001)
    # optim = torch.optim.SGD([tens], lr=0.0001, momentum=0.9)

    for j in range(niters):
        optim.zero_grad()

        loss = f(tens)

        loss.backward()
        optim.step()

        losses.append(loss.item())

    loss_table.append(losses)

    fig.add_trace(go.Scatter(y=losses, mode='markers+lines', name=str(i)))

fig = fig.update_layout(yaxis_type='log')

loss_df = pd.DataFrame(loss_table)[::-1]

In [9]:
fig.show()

In [ ]:
leq = modf.fun(X0, par, cond)

In [ ]:
sum_squares = np.sum(np.square(leq))

In [ ]:
sum_squares

In [ ]:
torch_fun(X0_tensor, par, cond)

In [ ]:
i=1

In [ ]:
torch.square(torch.abs(x[i]) - torch.abs(x[i-1]) - (rho/2)*(uin[i-1]**2-uin[i]**2 + (fin[i]*Ly_list[i-1]*uin[i]**2)/Din + Ky_in[i]*uin[i-1]**2 + K_se[i] * uin[N-1]**2))